Treinaremos nosso modelo ajustando os pesos várias vezes para fazer previsões cada vez melhores, utilizando uma técnica de otimização chamada gradiente de descida.

## Dados de treinamento

In [1]:
import numpy as np
import torch

Podemos representar os dados de treinamento utilizando duas matrizes: `inputs` e `targets`, cada uma com uma linha por observação e uma coluna por variável.

In [2]:
# Input (temperatura, chuva, umidade)
inputs = np.array(
    [[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]],
    dtype="float32",
)

In [3]:
# Targets (maçãs, laranjas)
targets = np.array(
    [[56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype="float32"
)

In [6]:
# Converte os inputs e targets para tensores
inputs_t = torch.from_numpy(inputs)
targets_t = torch.from_numpy(targets)

print(inputs)
print(targets)

[[ 73.  67.  43.]
 [ 91.  88.  64.]
 [ 87. 134.  58.]
 [102.  43.  37.]
 [ 69.  96.  70.]]
[[ 56.  70.]
 [ 81. 101.]
 [119. 133.]
 [ 22.  37.]
 [103. 119.]]


Os pesos e os viéses também podem ser representados por matrizes, inicializados como valores aleatórios por `randn`. A primeira linha de `w` e o primeiro elemento de `b` são usados para predizer a primeira variável-alvo.

In [7]:
# Pesos e bias
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.3512,  0.9042, -0.0087],
        [-0.1190, -0.2331, -0.2502]], requires_grad=True)
tensor([ 0.4272, -0.0864], requires_grad=True)


In [8]:
def model(x):
    return x @ w.t() + b

In [13]:
model(inputs_t)

tensor([[ 86.2708, -35.1483],
        [111.3979, -47.4392],
        [151.6387, -56.1843],
        [ 74.8055, -31.5035],
        [110.8539, -48.1874]], grad_fn=<AddBackward0>)